In [1]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

from sklearn.decomposition import TruncatedSVD

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model

In [7]:
import models
architecture = getattr(models, "Linear3NoBias") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    True,
    shuffle_train=False)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/Linear3NoBias/curve1/checkpoint-100.pt')['model_state'])
model2.load_state_dict(torch.load('curves/Linear3NoBias/curve2/checkpoint-100.pt')['model_state'])


Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [43]:
model1.cuda();
model1.eval();
test_model(model1, loaders, cuda=True)

train results {'nll': 0.4516510422611237, 'loss': 0.4516510422611237, 'accuracy': 84.766}
test results {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34}


({'nll': 0.4516510422611237, 'loss': 0.4516510422611237, 'accuracy': 84.766},
 {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34})

In [9]:
def get_model(W, architecture, bias=False, per_layer=True):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  

    if per_layer:
        for parameter, w in zip(model_sampled.parameters(), W):
            parameter.data.copy_(torch.from_numpy(w))
    else:
        for i, parameter in enumerate(model_sampled.parameters()):
            w = W[i//2]
            if i % 2 == 0:
                offset = 0
            N = parameter.data.shape[1]
            w_part = w[:, offset:offset+N]
            offset = N
            if i % 2 == 0:
                parameter.data.copy_(torch.from_numpy(w_part))
            else:
                parameter.data.copy_(torch.from_numpy(w_part.T))
            

    return model_sampled

def samples(model):
    p = [list(model.parameters())[i].data.cpu().numpy() for i in range(len(list(model.parameters())))]
    return p

def samples_per_layer(model, bias=True):
    p = samples(model)
    if bias:
        p = [np.hstack([p[i], p[i+1][:, None]]) for i in range(0, len(p), 2)]        
    return p

In [48]:
layer = 0
model1.cuda()
model1.eval()
functions = []
targ = []
for X, y in loaders['test']:
    functions.append(model1(X.cuda(), N=layer))
    targ.append(y)

print('l', functions[0].shape)
targ = np.concatenate(targ)
funcs = np.concatenate([p.cpu().data.numpy() for p in functions])

l torch.Size([1024, 6144])


In [56]:
def get_X(data='test'):
    dataset = []
    for X, y in loaders[data]:
        dataset.append(X)

    dataset = np.concatenate([p.view(-1,3*32*32).cpu().data.numpy() for p in dataset])
    
    return dataset

In [57]:
data = get_X()

In [58]:
data.shape

(10000, 3072)

In [44]:
funcs = get_funcs(model1, data='test')
funcs2  = get_funcs(model2, data='test')

# funcs2 = funcs2[:10000]

In [15]:
U_init, Sigma_init, V_init = np.linalg.svd(funcs.T)

In [26]:
comp = 6144
U, Sigma, V = U_init[:, :comp], Sigma_init[:comp], V_init[:comp]

In [119]:
def get_funcs(model1, data='test', layer=0):
    model1.cuda()
    model1.eval()
    functions = []
    for X, y in loaders[data]:
        functions.append(model1(X.cuda(), N=layer))

    funcs = np.concatenate([p.cpu().data.numpy() for p in functions])
    
    return funcs

def get_alpha(funcs, V):
    alpha = funcs.T @ V.T
    return alpha

def get_betas(funcs, V):
#     betas = np.linalg.inv(funcs.T @ V.T)
    f_inv = np.linalg.pinv(funcs.T)
    betas = V @ f_inv
    return betas

def accuracy(pred):
    ens_acc = 100.0 * np.mean(np.argmax(pred, axis=1) == targ)
    print(ens_acc)

In [91]:
alpha1 = get_alpha(funcs, V)
alpha2 = get_alpha(funcs2, V)

In [92]:
betas1 = get_betas(funcs, V)
betas2 =  get_betas(funcs2, V)

In [59]:
W10 = list(model1.parameters())[0].data.cpu().numpy()
W20 = list(model2.parameters())[0].data.cpu().numpy()

In [32]:
W11 = list(model1.parameters())[-2].data.cpu().numpy()
W21 = list(model2.parameters())[-2].data.cpu().numpy()

In [33]:
W12 = list(model1.parameters())[-1].data.cpu().numpy()
W22 = list(model2.parameters())[-1].data.cpu().numpy()

In [70]:
Wmodel1 = samples_per_layer(model1, bias=False)
Wmodel2 = samples_per_layer(model2, bias=False)

In [239]:
Wmodel = []
for i, (S1, S2) in enumerate(zip(Wmodel1, Wmodel2)):
    if i==0:
        print(S1.T.shape)
        cntr = Connector(S1.T, S2.T)
        S = cntr.arc_connect(t=0.5)[1].T
    else:
        print(S1.shape)
        cntr = Connector(S1, S2)
        S = cntr.arc_connect(t=0.5)[1]
    Wmodel.append(S)

(3072, 6144)
(2000, 6144)
(10, 2000)


In [240]:
model = get_model(Wmodel, architecture, bias=False, per_layer=True)
funcs3 =  get_funcs(model, data='test')

In [241]:
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

train results {'nll': 1.2921134415817261, 'loss': 1.2921134415817261, 'accuracy': 56.298}
test results {'nll': 1.6340929803848268, 'loss': 1.6340929803848268, 'accuracy': 42.44}


({'nll': 1.2921134415817261, 'loss': 1.2921134415817261, 'accuracy': 56.298},
 {'nll': 1.6340929803848268, 'loss': 1.6340929803848268, 'accuracy': 42.44})

In [242]:
alpha3 = get_alpha(funcs3, V) 
betas3 =  get_betas(funcs3, V)
# W31 = list(model.parameters())[-2].data.cpu().numpy()
# W31 = list(model.parameters())[-2].data.cpu().numpy()

In [243]:
pred = np.maximum(funcs3 @ betas3.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)

60.029999999999994


In [244]:
pred = np.maximum(funcs3 @ betas3.T @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)

60.19


In [245]:
pred = np.maximum(funcs2 @ betas2.T @ (alpha1.T @ W11.T + alpha2.T @ W21.T)/2, 0) @ (W12.T+W22.T)/2
accuracy(pred)

42.54


In [246]:
pred = np.maximum(funcs @ W11.T, 0) @ W12.T
accuracy(pred)
pred = np.maximum(funcs @ betas1.T @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)
pred = np.maximum(funcs2 @ betas2.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)
pred = np.maximum(funcs @ betas1.T @ alpha2.T @ W21.T, 0) @ W22.T
accuracy(pred)
pred = np.maximum(funcs2 @ betas2.T  @ alpha1.T @ W11.T, 0) @ W12.T
accuracy(pred)

60.34
60.34
60.099999999999994
60.050000000000004
60.12


In [222]:
g1 = get_funcs(model1, data='test', layer=1)
g2 = get_funcs(model2, data='test', layer=1)


In [247]:
Wh1 = alpha1.T @ W11.T
Wh2 = alpha2.T @ W21.T
cntr = Connector(Wh1.T, Wh2.T)
Wh = cntr.arc_connect(t=0.5)[1].T
print(Wh1.shape)

g3 = np.maximum(funcs2 @ betas2.T @ Wh, 0)

(6144, 2000)


In [224]:
U_init2, Sigma_init2, V_init2 = np.linalg.svd(g1.T)

In [225]:
U_init2.shape, Sigma_init2.shape, V_init2.shape

((2000, 2000), (2000,), (10000, 10000))

In [226]:
comp = 2000
U2, Sigma2, V2 = U_init2[:, :comp], Sigma_init2[:comp], V_init2[:comp]

In [227]:
alpha12 = get_alpha(g1, V2)
alpha22 = get_alpha(g2, V2)

betas12 = get_betas(g1, V2)
betas22 =  get_betas(g2, V2)


In [248]:
alpha32 = get_alpha(g3, V2) 
betas32 =  get_betas(g3, V2)

In [229]:
V2

array([[ 0.00974413,  0.01173222,  0.00887076, ...,  0.00287552,
         0.00735211,  0.00917149],
       [-0.00934512, -0.00041445, -0.00482961, ...,  0.00019445,
         0.00409823,  0.00771557],
       [ 0.00780636, -0.01256509, -0.00753748, ..., -0.00167393,
        -0.00991292, -0.00814729],
       ...,
       [-0.00226622,  0.00241323, -0.00196166, ..., -0.00138159,
         0.00022485,  0.00020743],
       [ 0.01009552, -0.00078211, -0.00042527, ..., -0.00056638,
        -0.00158174, -0.00494823],
       [ 0.00178276, -0.00328206, -0.00023848, ...,  0.0019458 ,
         0.00532567, -0.00387456]], dtype=float32)

In [230]:
g3 @ betas32.T

array([[ 0.01011109, -0.01019042,  0.00757564, ..., -0.00211355,
         0.00237924, -0.00440493],
       [ 0.01068594, -0.00065097, -0.01435278, ...,  0.00312119,
        -0.00088184,  0.00192975],
       [ 0.00805384, -0.00334248, -0.00803568, ...,  0.00328906,
         0.00528805, -0.00202743],
       ...,
       [ 0.00340781,  0.00052418, -0.00194526, ...,  0.00181662,
         0.00428702, -0.00453944],
       [ 0.0067509 ,  0.00476765, -0.01002657, ..., -0.00612088,
         0.00396887, -0.00070291],
       [ 0.01001711,  0.00791248, -0.00736805, ...,  0.00261683,
         0.00293513,  0.00477807]], dtype=float32)

In [231]:
g1 @ betas12.T

array([[ 0.00974413, -0.00934511,  0.00780636, ..., -0.00226623,
         0.01009551,  0.00178275],
       [ 0.01173222, -0.00041445, -0.01256509, ...,  0.00241323,
        -0.00078211, -0.00328206],
       [ 0.00887076, -0.00482961, -0.00753749, ..., -0.00196166,
        -0.00042527, -0.00023847],
       ...,
       [ 0.00287552,  0.00019444, -0.00167394, ..., -0.0013816 ,
        -0.00056638,  0.00194579],
       [ 0.00735211,  0.00409823, -0.00991292, ...,  0.00022486,
        -0.00158173,  0.00532567],
       [ 0.00917149,  0.00771557, -0.0081473 , ...,  0.00020742,
        -0.00494823, -0.00387455]], dtype=float32)

In [232]:
g2 @ betas22.T

array([[ 0.0101224 , -0.00952203,  0.0061153 , ..., -0.00277059,
         0.00187412,  0.00129607],
       [ 0.01069099, -0.00068937, -0.01286153, ...,  0.00157149,
        -0.00330083, -0.00549832],
       [ 0.00870076, -0.00488605, -0.00746369, ...,  0.00711501,
        -0.00533918,  0.00135434],
       ...,
       [ 0.00385828, -0.00038704, -0.00200862, ...,  0.00919093,
         0.00178433, -0.00742358],
       [ 0.00639355,  0.0043392 , -0.0106674 , ..., -0.00374491,
        -0.00068105,  0.00098946],
       [ 0.00950932,  0.00721175, -0.00737754, ...,  0.00190379,
        -0.00362128,  0.00741634]], dtype=float32)

In [249]:
pred = g3 @ betas32.T @ alpha12.T @ W12.T
accuracy(pred)

58.13


In [250]:
pred = g3 @ betas32.T @ alpha22.T @ W22.T
accuracy(pred)

58.440000000000005


In [235]:
pred = g3 @ betas32.T @ (alpha12.T @ W12.T + alpha22.T @ W22.T)/2
accuracy(pred)

59.13


In [208]:
pred = g1 @ betas12.T @ (alpha12.T @ W12.T + alpha22.T @ W22.T)/2
accuracy(pred)

60.86


In [204]:
pred = g1 @ betas12.T @ alpha12.T @ W12.T
accuracy(pred)

60.34


In [205]:
pred = g2 @ betas22.T @ alpha12.T @ W12.T
accuracy(pred)

59.95


In [206]:
pred = g2 @ betas22.T @ alpha22.T @ W22.T
accuracy(pred)

60.4


In [207]:
pred = g1 @ betas12.T @ alpha22.T @ W22.T
accuracy(pred)

59.699999999999996


In [5]:
def accuracy_svd(model, TSVD=None, data='train', N_comp=200, layer=0):
    
    model.cuda()
    model.eval()
    functions = []
    targ = []
    for X, y in loaders[data]:
        functions.append(model(X.cuda(), N=layer))
        targ.append(y)
        
    print('l', functions[0].shape)
    targ = np.concatenate(targ)
    funcs = np.concatenate([p.cpu().data.numpy() for p in functions])
    
    print(funcs.T[:1])
    
    if TSVD is None:
        TSVD = TruncatedSVD(n_components=N_comp, n_iter=5, random_state=1)
        TSVD.fit(funcs.T)
    funcs_PCA = TSVD.transform(funcs.T).T
    funcs_PCA_inv = TSVD.inverse_transform(funcs_PCA.T).T
    functions_PCA = [torch.FloatTensor(funcs_PCA_inv[i*1024:(i+1)*1024]).cuda() for i in range(len(functions))]
    
    pred = []
    for f in functions_PCA:
        pred.append(model.last_layers(f, N=layer+1).cpu().data.numpy())
    
    pred = np.concatenate(pred)

    ens_acc = 100.0 * np.mean(np.argmax(pred, axis=1) == targ)

    return ens_acc, TSVD, funcs_PCA


In [6]:
def accuracy_interpolation(model1, model2, funcs_PCA1, funcs_PCA2, TSVD=None, layer=0, t=0.5, func='arc_connect'):
    
    targ = []
    functions = 0
    for X, y in loaders['test']:
        functions+=1
        targ.append(y)
    targ = np.concatenate(targ)
    
    model1.cuda()
    model1.eval()
    model2.cuda()
    model2.eval()
    
    
    print('funcs_PCA', funcs_PCA1.shape)
    cntr = Connector(funcs_PCA1.T, funcs_PCA2.T)
    method = getattr(cntr, 'arc_connect')
    funcs_PCA = method(t=t)[1].T
    funcs_PCA_inv = TSVD.inverse_transform(funcs_PCA.T).T
    functions_PCA = [torch.FloatTensor(funcs_PCA_inv[i*1024:(i+1)*1024]).cuda() for i in range(functions)]
    
    pred = []
    ll1 = list(model1.parameters())[-1].cpu().data.numpy()
    ll2 = list(model2.parameters())[-1].cpu().data.numpy()
    print('ll', ll1.shape)
    cntr = Connector(ll1.T, ll2.T)
    method = getattr(cntr, func)
    ll = method(t=t)[1].T
    ll = torch.FloatTensor(ll.T).cuda()
    for f in functions_PCA:
#         print('ll, f', ll.shape, f.shape)
        p = f@ll
#         print('p', p.shape)
        pred.append(p.cpu().data.numpy())
    
    pred = np.concatenate(pred)

    ens_acc = 100.0 * np.mean(np.argmax(pred, axis=1) == targ)

    return ens_acc


In [7]:
layer = 1
acc, TSVD, funcs_PCA2 = accuracy_svd(model2, TSVD=None, data='test', N_comp=500, layer=layer)
print(acc)

l torch.Size([1024, 2000])
[[0. 0. 0. ... 0. 0. 0.]]
59.440000000000005


In [12]:
accuracy_interpolation(model1, model2, funcs_PCA1, funcs_PCA2, TSVD=TSVD, layer=layer, 
                       t=0.5, func='arc_connect')

funcs_PCA (500, 2000)
ll (10, 2000)


54.52

In [9]:
acc, _, funcs_PCA1 = accuracy_svd(model1, TSVD=TSVD, data='test', layer=layer)
print(acc)

l torch.Size([1024, 2000])
[[0. 0. 0. ... 0. 0. 0.]]
59.21999999999999


In [10]:
acc, _, _ = accuracy_svd(model2, TSVD=TSVD, data='test', layer=layer)
print(acc)

l torch.Size([1024, 2000])
[[0. 0. 0. ... 0. 0. 0.]]
59.440000000000005


In [38]:

model1.cuda()
functions = []
targ = []
for X, y in loaders['train']:
    functions.append(model1(X.cuda(), N=0))
    targ.append(y)
targ = np.concatenate(targ)

In [8]:
funcs = np.concatenate([p.cpu().data.numpy() for p in functions])

In [9]:
funcs.shape

(50000, 6144)

In [10]:
# from sklearn.decomposition import PCA
# transPSA = PCA(n_components=2000, svd_solver='full', random_state=1)
# U, Sigma, V = np.linalg.svd(funcs[:])



TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=1, tol=0.0)

In [11]:
funcs_PCA = TSVD.transform(funcs.T).T

In [12]:
funcs_PCA.shape
funcs_PCA_inv = TSVD.inverse_transform(funcs_PCA.T).T

In [13]:
funcs_PCA_inv.shape

(50000, 6144)

In [14]:
# funcs_PCA_inv = transPSA.inverse_transform(funcs_PCA)

In [15]:
# funcs

In [16]:
functions_PCA = [torch.FloatTensor(funcs_PCA_inv[i*1024:(i+1)*1024]).cuda() for i in range(len(functions))]

In [49]:
pred = []
for f in functions_PCA:
    pred.append(model1.last_layers(f, N=1).cpu().data.numpy())

In [50]:
pred = np.concatenate(pred)

In [51]:
ens_acc = 100.0 * np.mean(np.argmax(pred, axis=1) == targ)
print(ens_acc)


64.912


In [35]:
functions_test = []
targ_test = []
for X, y in loaders['test']:
    functions_test.append(model1(X.cuda(), N=0))
    targ_test.append(y)
    
functions_test = np.concatenate([p.cpu().data.numpy() for p in functions])
targ_test = np.concatenate(targ_test)

 
x = TSVD.transform(functions_test.cpu().data.numpy().T)
x = TSVD.inverse_transform(x).T
functions_PCA = [torch.FloatTensor(funcs_PCA_inv[i*1024:(i+1)*1024]).cuda() for i in range(len(functions))]

y_hat = model2.last_layers(torch.FloatTensor(x).cuda(), N=1).cpu().data.numpy()
pred_test.append(y_hat)
targ_test.append(y)
targ_test = np.concatenate(targ_test)
pred_test = np.concatenate(pred_test)


(6144, 1024)


ValueError: shapes (6144,1024) and (50000,100) not aligned: 1024 (dim 1) != 50000 (dim 0)

In [82]:
ens_acc = 100.0 * np.mean(np.argmax(pred_test, axis=1) == targ_test)
print(ens_acc)

28.449999999999996


In [28]:
model1.cuda();
model1.eval();
test_model(model1, loaders, cuda=True)

train results {'nll': 0.4526240879917145, 'loss': 0.4526240879917145, 'accuracy': 84.736}
test results {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34}


({'nll': 0.4526240879917145, 'loss': 0.4526240879917145, 'accuracy': 84.736},
 {'nll': 1.2863960428237915, 'loss': 1.2863960428237915, 'accuracy': 60.34})

In [37]:
# model2.cuda();
# model2.eval();
# test_model(model2, loaders, cuda=True)

In [38]:
def samples(model):
    p = [list(model.parameters())[i].data.cpu().numpy() for i in range(len(list(model.parameters())))]
    return p

def samples_per_layer(model, bias=True):
    p = samples(model)
    if bias:
        p = [np.hstack([p[i], p[i+1][:, None]]) for i in range(0, len(p), 2)]        
    return p

def samples_butterfly(model, bias=True):
    if bias:
        return None
    else:   
        p = samples_per_layer(model, bias=bias)
        parameters = [np.hstack([p[i], p[i+1].T]) for i in range(0, len(p), 2)]
    return parameters

def get_model(W, architecture, bias=False, per_layer=True):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  

    if per_layer:
        for parameter, w in zip(model_sampled.parameters(), W):
            parameter.data.copy_(torch.from_numpy(w))
    else:
        for i, parameter in enumerate(model_sampled.parameters()):
            w = W[i//2]
            if i % 2 == 0:
                offset = 0
            N = parameter.data.shape[1]
            w_part = w[:, offset:offset+N]
            offset = N
            if i % 2 == 0:
                parameter.data.copy_(torch.from_numpy(w_part))
            else:
                parameter.data.copy_(torch.from_numpy(w_part.T))
            

    return model_sampled

In [39]:
def transform(x1, x2, E12, E22_inv, second=False):
    y1 = x1 - E12 @ E22_inv @ x2
    return y1.T
   
def inv_transform(y1, y2, E12, E22_inv, second=False):
    x1 = y1 + E12 @ E22_inv @ y2
    return x1.T

In [40]:
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)

In [51]:
def connect_simple(W1, W2, lines=True, func='arc_connect'):
    
    if lines:    
        print('W1', W1.shape)
        cntr = Connector(W1, W2)
        f = getattr(cntr, func)
        p_res = f()[1]
    else:
        print('W1', W1.T.shape)
        cntr = Connector(W1.T, W2.T)
        f = getattr(cntr, func)
        p_res = f()[1].T
        
    return p_res

def connect_cov(W1, W2, P1, P2, P_f, lines=True, inverse=False,  func='arc_connect'):
        
        if inverse:
            P1, W1 = P1.T, W1.T
            P2, W2 = P2.T, W2.T
 
        p1 = np.hstack([P1, W1.T])
        p2 = np.hstack([P2, W2.T])
        len_x2 = len(W1)
        p = np.concatenate([p1,p2])
        
        print('p', p.shape)
        print('len', len_x2)
        
        mu1 = p.mean(0)[:-len_x2]
        mu2 = p.mean(0)[len_x2:]

        print('p', p.shape)

        p = p - p.mean(0)    
        cov = p.T @ p

        E12 = cov[:-len_x2, -len_x2:]
        E22 = cov[-len_x2:, -len_x2:]
        E22_inv = np.linalg.inv(E22)

        
        print(1, P1.T.shape, W1.shape)
        W1 = transform(P1.T, W1, E12,  E22_inv)
        W2 = transform(P2.T, W2, E12,  E22_inv)
        print('W1', W1.shape)
    
        p_res = connect_simple(W1, W2, lines=lines, func=func)
        
        print(2, p_res.T.shape, P_f.shape)
        p_res_tr = inv_transform(p_res.T, P_f, E12,  E22_inv)
        
        return p_res_tr

In [42]:
# parameters_res_tr = []
# parameters_res = []

# print(3)
# p_res = connect_simple(parameters1[0], parameters2[0], lines=False)
# parameters_res_tr.append(p_res)


# print(2)
# p_res = connect_simple(parameters1[-2], parameters2[-2], lines=False)
# parameters_res_tr.append(p_res)


# print(1)
# print('parameters', parameters2[-1].shape)
# # p_res = connect_simple(parameters1[-1], parameters2[-1], lines=True)
# p_res = connect_cov(parameters1[1], parameters2[1], parameters1[-1],  parameters2[-1], parameters_res_tr[-1].T, 
#                     lines=False, inverse=True).T


# parameters_res_tr.append(p_res)
    
# model = get_model(parameters_res_tr, architecture, per_layer=True)
# model.cuda();
# model.eval();
# test_model(model, loaders, cuda=True)

    

In [43]:
parameters1[0].shape

(6144, 3072)

In [44]:
p_mean = []
for i, (p1, p2) in enumerate(zip(parameters1, parameters2)):
    p = np.concatenate([p1.T,p2.T])
#     print(p.shape)
    mean = p.mean(0)
    p_mean.append(mean)
#     print(p1.T.shape, mean.shape)
    parameters1[i] = (p1.T - mean).T
    parameters2[i] = (p2.T - mean).T

In [23]:
p_mean = []
for i, (p1, p2) in enumerate(zip(parameters1, parameters2)):
    p = np.concatenate([p1,p2])
    print(p.shape)
    mean = p.mean(0)
    p_mean.append(mean)
    parameters1[i] = p1 - mean
    parameters2[i] = p2 - mean

(12288, 3072)
(4000, 6144)
(20, 2000)


In [24]:
# plt.hist(parameters1[0][:, 1], bins=100);

In [93]:
# parameters1[0] -= parameters1[0].mean(0)
# parameters1[0][:, 1].mean(), parameters1[0].T[:, 1].mean()

In [53]:
parameters1[0][:, 1].mean(), parameters1[0].T[:, 1].mean()

(0.006478533, -3.176718e-05)

In [48]:
np.vstack([ parameters2[-1],  parameters1[-1]]).shape

(20, 2000)

In [67]:
parameters_res_tr = []
parameters_res = []
func = 'arc_connect'

print(1)
print('parameters', parameters2[-1].shape)
p_res = connect_simple(parameters1[-1], parameters2[-1], lines=False, func=func)
parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

print(2)
print('parameters', parameters2[-2].shape)
p_res = connect_simple(parameters1[-2], parameters2[-2], lines=True, func=func)
# p_res = connect_cov(parameters1[-1], parameters2[-1], parameters1[-2],  parameters2[-2], parameters_res[0], 
#                     lines=False, func=func)
parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

print(3)
print('parameters', parameters2[0].shape)
p_res = connect_simple(parameters1[0], parameters2[0], lines=False, func=func)
# p_res = connect_cov(parameters1[1], parameters2[1], parameters1[0],  parameters2[0], parameters_res[0], 
#                     lines=False, func=func)


parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

for i, (p, m) in enumerate(zip(parameters_res_tr, p_mean)):
    parameters_res_tr[i] = (p.T + m).T

    
model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

    

1
parameters (10, 2000)
W1 (2000, 10)
2
parameters (2000, 6144)
W1 (2000, 6144)
3
parameters (6144, 3072)
W1 (3072, 6144)
train results {'nll': 1.271210627593994, 'loss': 1.271210627593994, 'accuracy': 57.922}
test results {'nll': 1.6456598905563355, 'loss': 1.6456598905563355, 'accuracy': 42.36}


({'nll': 1.271210627593994, 'loss': 1.271210627593994, 'accuracy': 57.922},
 {'nll': 1.6456598905563355, 'loss': 1.6456598905563355, 'accuracy': 42.36})

In [88]:
parameters_res_tr = []
parameters_res = []
func = 'arc_connect'


print(1)
p_res = connect_simple(parameters1[-1], parameters2[-1], lines=False, func=func)
parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

print(2)
p_res = connect_simple(parameters1[-2], parameters2[-2], lines=False, func=func)
# p_res = connect_cov(parameters1[-1], parameters2[-1], parameters1[-2],  parameters2[-2], parameters_res[0], 
#                     lines=False, func=func)
parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

print(3)
p_res = connect_simple(parameters1[0], parameters2[0], lines=False, func=func)
# p_res = connect_cov(parameters1[1], parameters2[1], parameters1[0],  parameters2[0], parameters_res[0], 
#                     lines=False, func=func)
print('parameters', parameters2[0].shape)

parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

# for i, (p, m) in enumerate(zip(parameters_res_tr, p_mean)):
#     parameters_res_tr[i] = p + m

    
model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

    

1
2
3
parameters (6144, 3072)
train results {'nll': 1.3164503594589234, 'loss': 1.3164503594589234, 'accuracy': 57.19}
test results {'nll': 1.6831079917907714, 'loss': 1.6831079917907714, 'accuracy': 41.08}


({'nll': 1.3164503594589234, 'loss': 1.3164503594589234, 'accuracy': 57.19},
 {'nll': 1.6831079917907714, 'loss': 1.6831079917907714, 'accuracy': 41.08})

In [15]:
# parameters1 = samples_butterfly(model1, bias=False)
# parameters2 = samples_butterfly(model2, bias=False)
# for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA', 'inverse_connect_PCA', 
#              'third_cumulant_connect']:
#     parameters_res = []
#     print(func)
#     for p1, p2 in zip(parameters1, parameters2):
#         cntr = Connector(p1, p2)
#         f = getattr(cntr, func)
#         if 'PCA' in func:
#             res = f(K=300)[1]
#         elif 'third_cumulant' in func:
#             res = f(K=100)[1] 
#         else:
#             res = f()[1]
#         parameters_res.append(res)

#     model = get_model(parameters_res, architecture, per_layer=False)
#     model.cuda();
#     model.eval();
#     test_model(model, loaders, cuda=True)

In [17]:
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
train results {'nll': 1.9955093453598023, 'loss': 1.9955093453598023, 'accuracy': 51.474}
test results {'nll': 2.162593546295166, 'loss': 2.162593546295166, 'accuracy': 29.76}
arc_connect
train results {'nll': 1.3757157013320922, 'loss': 1.3757157013320922, 'accuracy': 50.706}
test results {'nll': 1.6448731899261475, 'loss': 1.6448731899261475, 'accuracy': 42.03}
arc_connect_PCA
train results {'nll': 1.5449913298034668, 'loss': 1.5449913298034668, 'accuracy': 42.81}
test results {'nll': 1.722438893699646, 'loss': 1.722438893699646, 'accuracy': 36.14}


In [18]:
# columns
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1.T, p2.T)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
        parameters_res.append(res.T)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

arc_connect
train results {'nll': 1.3138795108032226, 'loss': 1.3138795108032226, 'accuracy': 57.49}
test results {'nll': 1.6831079917907714, 'loss': 1.6831079917907714, 'accuracy': 41.08}
arc_connect_PCA
train results {'nll': 1.3796758843994141, 'loss': 1.3796758843994141, 'accuracy': 55.522}
test results {'nll': 1.7231661840438843, 'loss': 1.7231661840438843, 'accuracy': 39.77}


In [19]:
# one last column and lines
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for ind, (p1, p2) in enumerate(zip(parameters1, parameters2)):
        if ind==len(parameters1)-1:
            print('last')
            p1, p2 = p1.T, p2.T
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        if ind==len(parameters1)-1:
            res = res.T
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

arc_connect
last
train results {'nll': 1.402382646522522, 'loss': 1.402382646522522, 'accuracy': 49.57}
test results {'nll': 1.6654330642700195, 'loss': 1.6654330642700195, 'accuracy': 41.36}
arc_connect_PCA
last
train results {'nll': 1.4344119925308227, 'loss': 1.4344119925308227, 'accuracy': 48.944}
test results {'nll': 1.6590761445999145, 'loss': 1.6590761445999145, 'accuracy': 41.85}


In [20]:
# all columns except first lines
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for ind, (p1, p2) in enumerate(zip(parameters1, parameters2)):
        if ind!=0:
            print('last')
            p1, p2 = p1.T, p2.T
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        if ind!=0:
            res = res.T
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

arc_connect
last
last
train results {'nll': 1.3678412181472779, 'loss': 1.3678412181472779, 'accuracy': 53.644}
test results {'nll': 1.6921545532226563, 'loss': 1.6921545532226563, 'accuracy': 40.73}
arc_connect_PCA
last
last
train results {'nll': 1.4357195449066162, 'loss': 1.4357195449066162, 'accuracy': 51.714}
test results {'nll': 1.7344378902435302, 'loss': 1.7344378902435302, 'accuracy': 39.71}
